In [1]:
# เรียกใช้ library สำหรับทำงานประกอบด้วย numpy pandas seaborn เเละ matplotlib
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

Using TensorFlow backend.
C:\Users\Anwa\Downloads\machine\Newfolder\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Anwa\Downloads\machine\Newfolder\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Anwa\Downloads\machine\Newfolder\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Anwa\Downloads\machi

In [2]:
# อ่านข้อมูลไฟล์ csv จาก https://raw.githubusercontent.com/nawinpoyen7/MachineLearning/main/DataSet/DataChange.csv
# และเเสดง 5 ตัวอย่างเเรก เพื่อตรวจสอบข้อมูล
url='https://raw.githubusercontent.com/nawinpoyen7/MachineLearning/main/DataSet/CombDataSet.csv'
dataset=pd.read_csv(url)
dataset.head(5)

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,0.015047,0.000000,0.005260,0.000000,0.003284,0.002718,0.000000,0.012819,0.007649,D
1,0.000000,0.014676,0.009085,0.014188,0.009852,0.002718,0.012093,0.009614,0.007649,A
2,0.000000,0.014676,0.016018,0.014188,0.009852,0.002718,0.000000,0.003205,0.012749,B
3,0.015047,0.014676,0.016018,0.014188,0.016421,0.000000,0.024186,0.006410,0.012749,B
4,0.000000,0.014676,0.009563,0.014188,0.006568,0.002718,0.024186,0.019229,0.012749,A


In [3]:
dataset.Segmentation.value_counts()

D    2268
A    1972
C    1970
B    1858
Name: Segmentation, dtype: int64

In [4]:
dataset.columns

Index(['Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1',
       'Segmentation'],
      dtype='object')

In [5]:
#Vals=['Gender','Ever_Married','Age','Graduated','Profession','Work_Experience','Spending_Score','Family_Size','Var_1']
#sns.pairplot(data=dataset,hue='Segmentation');

In [6]:
from sklearn.preprocessing import StandardScaler # ปรับช่วงข้อมูลด้วยวิธี standardization
from sklearn.preprocessing import OneHotEncoder
 # กำหนดตัวเเปร X เพื่อเก็บข้อมูลดอกไอริสสำหรับ features ที่ต้องการ 
Vals=['Profession', 'Spending_Score', 'Age', 'Graduated']
X=dataset[Vals] 
# สร้าง object ชื่อ scaler จาก class ของ StandardScaler เพื่อใช้สำหรับการเรียกใช้ในการปรับช่วงข้อมูล
scaler = StandardScaler()  
# ปรับช่วงข้อมูลเเละเก็บอยู่ในตัวเเปร X_scale ซึ่งข้อมูลจะอยู่ในรูปของ numpy array
X_scale=scaler.fit_transform(X)  
# ปรับให้เเสดงด้วยตัวเลขทศนิยม 4 ตำเเหน่ง 
X_scale = X_scale.round(4) 
y = np.array(dataset['Segmentation'])

encoder = OneHotEncoder()
y_targets = encoder.fit_transform(dataset[['Segmentation']]).toarray()

In [7]:
from sklearn.model_selection import train_test_split
test_size=0.3
X_train, X_test, y_train, y_test = train_test_split(
    X, y_targets, 
    test_size=test_size, random_state=7)

In [8]:
model = Sequential()

# สร้าง hidden layer ที่ 1 ประกอบด้วย 100 โหนด ซึ่งเชื่อมต่อกับ Input layer ประกอบด้วย 4 โหนด 
# เเละเลือกใช้ ReLu เป็น activation function 
model.add(Dense(100, input_dim=4, activation='relu'))

# สร้าง hidden layer ที่ 2 ประกอบด้วย 100 โหนด ซึ่งเชื่อมต่อกับ hidden layer ที่ 1 ประกอบด้วย 100 โหนด 
# เเละเลือกใช้ ReLu เป็น activation function 
model.add(Dense(100, input_dim=100, activation='relu'))

# สร้าง hidden layer ที่ 3 ประกอบด้วย 100 โหนด ซึ่งเชื่อมต่อกับ hidden layer ที่ 2 ประกอบด้วย 100 โหนด 
# เเละเลือกใช้ ReLu เป็น activation function
model.add(Dense(100, input_dim=100, activation='relu'))

# สร้าง hidden layer ที่ 4 ประกอบด้วย 100โหนด ซึ่งเชื่อมต่อกับ hidden layer ที่ 3 ประกอบด้วย 100 โหนด 
# เเละเลือกใช้ ReLu เป็น activation function
model.add(Dense(100, input_dim=100, activation='relu'))

# สร้าง output layer ประกอบด้วย 4 โหนด ซึ่งเชื่อมต่อกับ hidden layer ที่ 4 ประกอบด้วย 100 โหนด 
# เเละเลือกใช้ sigmoid เป็น activation function 
model.add(Dense(4, activation='sigmoid'))

In [9]:
# กำหนดให้ optimizer ใช้ method adam ในการปรับปรุงค่าด้วย learning rate = 0.005
# from keras.optimizers import Adam
# optimizer = Adam(learning_rate=0.01)
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
model.fit(X_train, y_train, epochs=100, verbose=1)


Epoch 1/100
5647/5647 [==============================] - 2s 331us/step - loss: 0.1940 - accuracy: 0.2690
Epoch 2/100
5647/5647 [==============================] - 3s 505us/step - loss: 0.1870 - accuracy: 0.2865
Epoch 3/100
5647/5647 [==============================] - 2s 330us/step - loss: 0.1746 - accuracy: 0.4011
Epoch 4/100
5647/5647 [==============================] - 2s 347us/step - loss: 0.1666 - accuracy: 0.4390
Epoch 5/100
5647/5647 [==============================] - 1s 263us/step - loss: 0.1656 - accuracy: 0.4434
Epoch 6/100
5647/5647 [==============================] - 1s 251us/step - loss: 0.1646 - accuracy: 0.4443
Epoch 7/100
5647/5647 [==============================] - 2s 290us/step - loss: 0.1654 - accuracy: 0.4466
Epoch 8/100
5647/5647 [==============================] - 2s 384us/step - loss: 0.1642 - accuracy: 0.4500
Epoch 9/100
5647/5647 [==============================] - 2s 310us/step - loss: 0.1647 - accuracy: 0.4468
Epoch 10/100
5647/5647 [==============================

In [11]:
results = model.evaluate(X_test, y_test)
print('Accuracy: %.3f' % results[1])

2421/2421 [==============================] - 0s 111us/step
Accuracy: 0.523
